In [47]:
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
regr = joblib.load("model/train_model.m")

In [48]:
ans = {}

In [49]:
import pandas as pd
import numpy as np
bj_now = pd.read_csv('submission/5.9/bjnow.csv')
bj_now = bj_now.fillna(method = 'ffill')
bj_now = bj_now.fillna(method = 'bfill')
for i,j in bj_now.groupby(bj_now['station_id']):
    print(j.iloc[-48:])
    print(i)
    x = np.hstack(np.array(j.iloc[-48:][['PM25_Concentration','PM10_Concentration','NO2_Concentration','CO_Concentration','O3_Concentration', 'SO2_Concentration' ]]))
    print(x)
    y = regr.predict(x.reshape(1,-1))
    ans[i] = y

     Unnamed: 0       id       station_id                 time  \
820       28389  2988135  aotizhongxin_aq  2018-05-06 18:00:00   
821       28424  2988191  aotizhongxin_aq  2018-05-06 19:00:00   
822       29961  2990826  aotizhongxin_aq  2018-05-06 20:00:00   
823       28459  2988267  aotizhongxin_aq  2018-05-06 21:00:00   
824       28494  2988323  aotizhongxin_aq  2018-05-06 22:00:00   
825       28529  2988379  aotizhongxin_aq  2018-05-06 23:00:00   
826       28564  2988435  aotizhongxin_aq  2018-05-07 00:00:00   
827       28599  2988491  aotizhongxin_aq  2018-05-07 01:00:00   
828       30030  2990896  aotizhongxin_aq  2018-05-07 02:00:00   
829       28634  2988567  aotizhongxin_aq  2018-05-07 03:00:00   
830       28669  2988623  aotizhongxin_aq  2018-05-07 04:00:00   
831       28704  2988679  aotizhongxin_aq  2018-05-07 05:00:00   
832       28739  2988735  aotizhongxin_aq  2018-05-07 06:00:00   
833       28774  2988791  aotizhongxin_aq  2018-05-07 07:00:00   
834       

donggaocun_aq
[ 41.  126.   20.    0.4 102.    8.   46.  117.   19.    0.4  92.    8.
  37.   98.   18.    0.4  84.    7.   40.   97.   18.    0.4  86.    7.
  42.  107.   16.    0.4  86.    6.   34.  113.   18.    0.4  63.    5.
  53.  106.   30.    0.4  37.    6.   44.   46.   34.    0.4  60.    6.
  26.   62.   16.    0.4  93.    6.   34.  117.   18.    0.7 104.   11.
  36.  123.   21.    1.  121.   18.   64.  148.   27.    1.6 148.   28.
  89.  161.   31.    2.1 174.   31.   72.  118.   23.    1.1 173.   31.
  48.  118.   17.    0.6 172.   15.   47.  110.   15.    0.4 162.   11.
  30.   82.   14.    0.4 148.    8.   24.   89.   16.    0.4 117.    6.
  23.   94.   25.    0.3  89.    5.   18.   73.   31.    0.3  68.    5.
  18.   50.   30.    0.3  64.    5.    9.   42.   26.    0.3  69.    5.
  19.   42.   21.    0.4  79.    7.   23.   56.   20.    0.4  73.    5.
  18.   34.   16.    0.4  73.    3.   22.   39.   17.    0.4  65.    2.
  17.   33.   17.    0.4  65.    2.    9.   33.   

In [50]:
ld_regr = joblib.load("model/ld_train_model.m")

In [51]:
ld_now = pd.read_csv('submission/5.9/ldnow.csv')
ld_now = ld_now.fillna(method = 'ffill')
ld_now = ld_now.fillna(method = 'bfill')
for i,j in ld_now.groupby(ld_now['station_id']):
    print(i)
    x = np.hstack(np.array(j.iloc[-48:][['PM25_Concentration','PM10_Concentration','NO2_Concentration']]))
    y = ld_regr.predict(x.reshape(1,-1))
    ans[i] = y

BL0
BX1
BX9
CD1
CD9
CT2
CT3
GN0
GN3
GR4
GR9
HV1
KF1
LW2
MY7
RB7
ST5
TD5
TH4


In [52]:
submit2histdata={} 
for i in list(ans.keys()):
    submit2histdata[i] = i
submit2histdata['wanshouxig_aq'] = 'wanshouxigong_aq'
submit2histdata['aotizhongx_aq'] = 'aotizhongxin_aq'
submit2histdata['nongzhangu_aq'] = 'nongzhanguan_aq'
submit2histdata['fengtaihua_aq'] = 'fengtaihuayuan_aq'
submit2histdata['miyunshuik_aq'] = 'miyunshuiku_aq'
submit2histdata['yongdingme_aq'] = 'yongdingmennei_aq'
submit2histdata['xizhimenbe_aq'] = 'xizhimenbei_aq'

In [53]:
def split_id(input):
    item,num = input.split('#')
    return item,num

In [54]:
submit_csv = pd.read_csv('data/sample_submissioin.csv')
submit_data = np.array(submit_csv)
for i in range(len(submit_csv)):
    item,num = split_id(submit_csv.iloc[i].test_id)
    try:
        if(ans[submit2histdata[item]].shape[1]>200):
            submit_data[i][1:4]=ans[submit2histdata[item]].reshape(48,6)[int(num)][0:3]
        else:
            submit_data[i][1:4]=ans[submit2histdata[item]].reshape(48,3)[int(num)]
    except:
        pass
submit = pd.DataFrame(submit_data,columns=submit_csv.columns,index=submit_csv.test_id)
submit = submit.drop('test_id',1)
submit[submit<0] = 0
submit.to_csv('submission/5.9/submit9.csv')

In [10]:
submit_csv.data

AttributeError: 'DataFrame' object has no attribute 'data'

In [11]:
submit = pd.DataFrame(submit_data,columns=submit_csv.columns)

In [12]:
# submit = submit.drop('test_id',1)
submit.to_csv('first_submit.csv')

In [13]:
submit 

,test_id,PM2.5,PM10,O3
0,dongsi_aq#0,53.4239,86.99,52.1773
1,dongsi_aq#1,54.7588,89.4748,57.5728
2,dongsi_aq#2,56.995,87.8857,59.359
3,dongsi_aq#3,52.6974,85.7024,57.2122
4,dongsi_aq#4,50.157,89.183,59.405
5,dongsi_aq#5,47.4357,91.4457,59.3431
6,dongsi_aq#6,49.2416,92.5929,60.402
7,dongsi_aq#7,42.8541,88.0301,56.4292
8,dongsi_aq#8,38.0757,83.6106,52.983
9,dongsi_aq#9,34.8811,80.1837,50.8976


In [14]:
submit.to_csv('submission/5.10/first_submit.csv')

In [37]:
submit_csv.iloc[2]

test_id    dongsi_aq#2
PM2.5                0
PM10                 0
O3                   0
Name: 2, dtype: object

In [35]:
submit_csv.iloc[2]

test_id    dongsi_aq#2
PM2.5                0
PM10                 0
O3                   0
Name: 2, dtype: object

In [25]:
ans['BL0'].reshape(48,3)[2]

array([16.36237681, 27.170711  , 55.85489369])

In [10]:
import DataSet
mydata = DataSet.DataSet('data/london_aq.csv')
all_ids = mydata.get_all_ids()

In [11]:
all_ids

['BL0',
 'CD1',
 'CD9',
 'GN0',
 'GN3',
 'GR4',
 'GR9',
 'HV1',
 'KF1',
 'LW2',
 'MY7',
 'ST5',
 'TH4']

In [9]:
mydata.get_data('BL0','2017-02-01','2018-02-01')

,Unnamed: 0,utc_time,station_id,PM2.5,PM10,NO2
utc_time,,,,,,
2017-02-01 00:00:00,744,2017/2/1 0:00,BL0,7.2,9.5,34.6
2017-02-01 01:00:00,745,2017/2/1 1:00,BL0,6.8,8.3,33.0
2017-02-01 02:00:00,746,2017/2/1 2:00,BL0,7.3,9.4,32.4
2017-02-01 03:00:00,747,2017/2/1 3:00,BL0,13.4,16.8,33.4
2017-02-01 04:00:00,748,2017/2/1 4:00,BL0,20.5,20.1,33.8
2017-02-01 05:00:00,749,2017/2/1 5:00,BL0,21.0,21.3,30.7
2017-02-01 06:00:00,750,2017/2/1 6:00,BL0,16.9,17.1,39.3
2017-02-01 07:00:00,751,2017/2/1 7:00,BL0,14.0,14.6,46.7
2017-02-01 08:00:00,752,2017/2/1 8:00,BL0,9.4,10.3,47.4
